Przykład 4
======
Znając wynik poprzedniego przykładu spróbujemy napisać klasyfikator, który na podstawie tego ile użytkownik spędza czasu na
stronie, powie czy to jest prawdziwy klient, czy fraudster.

Skorzystamy z naiwnego Bayesa.
To czego szukamy to wielkości $P(F|t)$ oraz $P(C|t)$, czyli prawdopodobieństwo tego, że mamy do czynienia z fraudsterem/klientem biorąc pod uwagę to ile czasu spędził na stronie.
To co mamy to $P(t|F)$ i $P(t|C)$. Te wielkości można wyznaczyć za pomocą znormalizowanych histogramów z poprzedniego rozkładu. Funkcja, wyznaczająca prawdowpodobieństwo będzie wyglądać w ten sposób:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
dane = np.load("example3_data.npz")
frauds = dane['frauds']
clients = dane['clients']
def estimate_prob(t, dist, bins=1000):
    vals, bins_ = np.histogram(dist, bins=bins, normed=True)
    idx = np.searchsorted(bins_, t)
    dx = bins_[1] - bins_[0]
    return vals[idx]*dx

Funkcja przyjmuje, że prawdopodobieństwo spędzenia takiego czasu na stronie jest równe polu powierzchni odpowiedniego binu (słupka) znormalizowanego histogrmu.

Twierdzenie Bayesa pozwala wyznaczyć szukane prawdopodobieństwo: $P(F|t) = P(t|F)P(F)/P(t)$ oraz $P(C|t) = P(t|C)P(C)/P(t)$.
Ponieważ interesuje nas tylko które prawdopodobieństwo jest większe, wyznaczanie $P(t)$ nie jest konieczne. Wielkości $P(C)$ oraz $P(F)$, zwane zaczątkami można albo estymować na podstawie danych, albo na podstawie wcześniejszej wiedzy (na przykład zakładając, że fraudsterzy to 5% całego ruchu).

In [6]:
def bayes(t, distributions, bins):
    p_t1 = estimate_prob(t, distributions[0], bins[0])
    p_t2 = estimate_prob(t, distributions[1], bins[1])
    N1 = len(distributions[0])
    N2 = len(distributions[1])
    p1 = N1/(N1 + N2)
    p2 = 1 - p1
    # 0 oznacza wybór pierwszej klasy, 1 oznacza wybór drugiej
    return int(p_t1*p1 < p_t2*p2)

#TEST
# Sprwadzamy co wyznaczy klasyfikatro dla jakich czasow
test_t = np.arange(0, 2e6, 1000)
res = []
for t in test_t:
    res_ = bayes(t, (frauds, clients), (500, 500))
    res.append(res_)
    if not res_:
        print(t)
print(res.count(0), res.count(1))

1257000.0
1258000.0
1259000.0
1260000.0
1261000.0
1262000.0
1263000.0
1264000.0
1265000.0
1266000.0
1473000.0
1474000.0
1475000.0
1640000.0
1641000.0
1642000.0
1643000.0
1644000.0
1645000.0
1646000.0
1647000.0
1648000.0
1649000.0
1748000.0
1749000.0
1750000.0
1751000.0
1752000.0
1753000.0
1754000.0
1755000.0
1756000.0
1757000.0
1768000.0
1769000.0
1770000.0
1771000.0
1772000.0
1773000.0
1774000.0
1775000.0
1776000.0
1777000.0
1807000.0
1808000.0
1809000.0
1810000.0
1811000.0
1812000.0
1813000.0
1814000.0
1815000.0
1816000.0
1837000.0
1838000.0
1839000.0
1840000.0
1841000.0
1842000.0
1843000.0
1844000.0
1845000.0
1846000.0
1906000.0
1907000.0
1908000.0
1909000.0
1910000.0
1911000.0
1912000.0
1913000.0
1914000.0
1931000.0
1932000.0
1933000.0
1934000.0
1955000.0
1956000.0
1957000.0
1958000.0
1959000.0
1960000.0
1961000.0
1962000.0
1963000.0
1964000.0
1965000.0
1966000.0
1967000.0
1968000.0
1969000.0
1970000.0
1971000.0
1972000.0
1973000.0
95 1905


Jak widać klasyfikator nie sprawdza się zbyt dobrze. 0 oznacza, że klasyfikator wybrał klienta, 1 oznacza, że klasyfikator wybrał fraudstera.